# 🎮 RL 완전 쉽게 시작하기: 1차원 보물찾기

## 목표
- 복잡한 수학 없이 RL의 핵심 개념 체험
- 직관적인 예제로 Q-learning 이해
- 01_rl_basics.ipynb를 위한 워밍업

## 📚 강화학습이 뭔가요?

**한 줄 요약**: 시행착오를 통해 배우는 AI

일상 예시:
- 🎮 게임: 죽으면서 배우는 다크소울
- 🚗 운전: 연습하면서 실력이 늘어감
- 🍳 요리: 맛없으면 다음엔 다르게 해봄

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

print("🚀 준비 완료! 이제 시작해봅시다!")

## 1️⃣ 초간단 세계 만들기 (1차원!)

복잡한 2D 격자 대신, 일직선 세계부터!

In [ ]:
# 우리의 작은 세계 (5칸짜리)
world = ['🏠', '⬜', '🕳️', '⬜', '💎']
#        시작   빈칸   함정   빈칸  보물

# 시각화 함수
def show_world(position):
    """현재 위치를 보여주는 함수"""
    display = []
    for i, cell in enumerate(world):
        if i == position:
            display.append(f'[😊{cell}]')
        else:
            display.append(f' {cell} ')
    print(' '.join(display))
    print(' 0    1    2    3    4  (위치 번호)')

print("🗺️ 우리의 게임 맵:")
show_world(0)
print("\n목표: 🏠에서 출발해서 💎을 찾아가세요!")
print("주의: 🕳️ 함정을 피하세요!")

## 2️⃣ 게임 규칙 정하기

In [ ]:
# 행동: 정말 간단하게 2개만!
actions = {
    0: '⬅️ 왼쪽',
    1: '➡️ 오른쪽'
}

# 보상 시스템 (점수)
rewards = {
    '🏠': 0,     # 시작점: 0점
    '⬜': -1,    # 빈칸: -1점 (빨리 가도록)
    '🕳️': -10,  # 함정: -10점 (피해야 함!)
    '💎': +10    # 보물: +10점 (목표!)
}

print("🎮 게임 규칙:")
print("\n행동:")
for key, value in actions.items():
    print(f"  {key}: {value}")

print("\n보상:")
for symbol, reward in rewards.items():
    print(f"  {symbol}: {reward:+3d}점")

## 3️⃣ 에이전트 만들기 (랜덤부터)

먼저 아무것도 모르는 에이전트를 만들어봅시다.

In [ ]:
def random_agent_play():
    """랜덤하게 행동하는 에이전트"""
    position = 0  # 시작 위치
    total_reward = 0
    path = [position]
    
    print("🎲 랜덤 에이전트의 여정:")
    print("-" * 30)
    
    for step in range(10):  # 최대 10걸음
        # 랜덤 행동 선택
        action = np.random.randint(2)
        
        # 이동
        if action == 0:  # 왼쪽
            new_position = max(0, position - 1)
        else:  # 오른쪽
            new_position = min(4, position + 1)
        
        # 보상 받기
        reward = rewards[world[new_position]]
        total_reward += reward
        
        print(f"Step {step+1}: {world[position]} → {actions[action]} → {world[new_position]} (보상: {reward:+d})")
        
        position = new_position
        path.append(position)
        
        # 목표 도달 또는 함정
        if position == 4:  # 보물 찾음!
            print("\n🎉 성공! 보물을 찾았습니다!")
            break
        elif position == 2:  # 함정에 빠짐
            print("\n😱 함정에 빠졌습니다!")
            break
    
    print(f"\n총 보상: {total_reward:+d}점")
    return total_reward, path

# 랜덤 에이전트 실행
reward, path = random_agent_play()

## 4️⃣ Q-Learning: 경험으로 배우기

### Q-테이블이란?
각 위치에서 각 행동이 얼마나 좋은지 점수를 매긴 표

예: Q[0][1] = 시작점(0)에서 오른쪽(1)으로 가는 것의 가치

In [ ]:
# Q-테이블 초기화
Q = np.zeros((5, 2))  # 5개 위치, 2개 행동

def show_q_table(Q):
    """Q-테이블을 예쁘게 출력"""
    print("\n📊 Q-테이블 (각 상태-행동의 가치):")
    print("위치     ⬅️왼쪽   ➡️오른쪽")
    print("-" * 30)
    for i, symbol in enumerate(world):
        print(f"{i}:{symbol}    {Q[i][0]:6.2f}   {Q[i][1]:6.2f}")

show_q_table(Q)
print("\n💡 처음엔 모든 값이 0 (아무것도 모름)")

## 5️⃣ 학습 시작! (100번 연습)

### 핵심 공식 (겁내지 마세요!)
```
새로운 지식 = 기존 지식 + 학습률 × (새로 배운 것 - 기존 지식)
```

In [ ]:
# 하이퍼파라미터 (조절 가능한 설정값)
learning_rate = 0.1   # 새 정보를 얼마나 빨리 받아들일까? (0~1)
discount = 0.9        # 미래 보상을 얼마나 중요하게 볼까? (0~1)
epsilon = 0.2         # 얼마나 자주 새로운 시도를 할까? (0~1)

print("🎓 Q-Learning 시작!")
print(f"학습률: {learning_rate}, 할인율: {discount}, 탐험율: {epsilon}")
print("\n100번 게임을 하면서 배워봅시다...\n")

# 학습 기록
episode_rewards = []

for episode in range(100):
    position = 0  # 시작 위치
    total_reward = 0
    
    # 한 에피소드 (한 번의 게임)
    for step in range(20):  # 최대 20걸음
        # ε-greedy 행동 선택
        if np.random.random() < epsilon:
            action = np.random.randint(2)  # 탐험: 랜덤
        else:
            action = np.argmax(Q[position])  # 활용: 최선의 선택
        
        # 행동 실행
        old_position = position
        if action == 0:  # 왼쪽
            position = max(0, position - 1)
        else:  # 오른쪽
            position = min(4, position + 1)
        
        # 보상 받기
        reward = rewards[world[position]]
        total_reward += reward
        
        # Q-테이블 업데이트 (핵심!)
        if position == 4:  # 목표 도달
            Q[old_position][action] += learning_rate * (reward - Q[old_position][action])
            break
        elif position == 2:  # 함정
            Q[old_position][action] += learning_rate * (reward - Q[old_position][action])
            break
        else:
            # Q-learning 공식
            best_next_action = np.max(Q[position])
            Q[old_position][action] += learning_rate * (
                reward + discount * best_next_action - Q[old_position][action]
            )
    
    episode_rewards.append(total_reward)
    
    # 진행 상황 출력
    if (episode + 1) % 20 == 0:
        avg_reward = np.mean(episode_rewards[-20:])
        print(f"에피소드 {episode+1:3d}: 평균 보상 = {avg_reward:+.2f}")

print("\n✅ 학습 완료!")

## 6️⃣ 학습 결과 확인

In [ ]:
# 학습된 Q-테이블 보기
show_q_table(Q)

print("\n💡 해석:")
print("- 양수(+): 좋은 행동")
print("- 음수(-): 나쁜 행동")
print("- 큰 값: 더 확실한 선택")

In [ ]:
# 학습 곡선 그리기
plt.figure(figsize=(10, 4))
plt.plot(episode_rewards, alpha=0.5, label='에피소드별 보상')
plt.plot(np.convolve(episode_rewards, np.ones(10)/10, mode='valid'), 
         linewidth=2, label='이동 평균 (10 에피소드)')
plt.xlabel('에피소드')
plt.ylabel('총 보상')
plt.title('학습 진행 상황')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("📈 보상이 점점 증가 = 학습이 되고 있음!")

## 7️⃣ 학습된 에이전트 테스트

In [ ]:
def smart_agent_play(Q, visualize=True):
    """학습된 Q-테이블을 사용하는 똑똑한 에이전트"""
    position = 0
    path = []
    total_reward = 0
    
    print("🤖 학습된 에이전트의 최적 경로:")
    print("=" * 40)
    
    for step in range(10):
        if visualize:
            clear_output(wait=True)
            print(f"\nStep {step}:")
            show_world(position)
            time.sleep(0.5)
        
        # Q-테이블에서 최선의 행동 선택
        action = np.argmax(Q[position])
        
        # 이동
        old_position = position
        if action == 0:
            position = max(0, position - 1)
        else:
            position = min(4, position + 1)
        
        path.append((old_position, action, position))
        reward = rewards[world[position]]
        total_reward += reward
        
        if position == 4:
            if visualize:
                clear_output(wait=True)
                print(f"\nStep {step+1}:")
                show_world(position)
            print("\n🎉 성공! 보물을 찾았습니다!")
            break
    
    print(f"\n경로:")
    for old_pos, act, new_pos in path:
        print(f"  {world[old_pos]} → {actions[act]} → {world[new_pos]}")
    
    print(f"\n총 보상: {total_reward:+d}점")
    print(f"총 걸음: {len(path)}걸음")
    
    return path

# 실행!
path = smart_agent_play(Q, visualize=False)

## 8️⃣ 핵심 개념 정리

### RL의 5가지 핵심 요소

In [ ]:
concepts = {
    "1. Agent (에이전트)": "보물을 찾는 😊 (의사결정자)",
    "2. Environment (환경)": "5칸짜리 세계 🏠⬜🕳️⬜💎",
    "3. State (상태)": "현재 위치 (0~4)",
    "4. Action (행동)": "왼쪽/오른쪽 이동 ⬅️➡️",
    "5. Reward (보상)": "각 위치의 점수 (+10, -10, -1, 0)"
}

print("🎯 강화학습 핵심 개념:")
print("=" * 40)
for concept, explanation in concepts.items():
    print(f"{concept:20s} = {explanation}")

print("\n📚 Q-Learning이란?")
print("- Q(상태, 행동) = 그 상태에서 그 행동의 예상 가치")
print("- 경험을 통해 Q값을 업데이트")
print("- 최종적으로 각 상태에서 최선의 행동을 알게 됨")

## 9️⃣ 실험해보기

### 🧪 실험 1: 함정을 더 위험하게 만들면?

In [ ]:
# 함정 패널티를 -50으로 변경
rewards_extreme = rewards.copy()
rewards_extreme['🕳️'] = -50

print("🔥 극한 모드: 함정 = -50점")
print("이제 에이전트가 어떻게 행동할까요?")

# 새로운 Q-테이블로 학습
Q_extreme = np.zeros((5, 2))

# 빠른 학습 (50 에피소드만)
for episode in range(50):
    position = 0
    for step in range(20):
        if np.random.random() < 0.1:  # 적은 탐험
            action = np.random.randint(2)
        else:
            action = np.argmax(Q_extreme[position])
        
        old_position = position
        if action == 0:
            position = max(0, position - 1)
        else:
            position = min(4, position + 1)
        
        reward = rewards_extreme[world[position]]
        
        if position in [2, 4]:  # 함정 또는 목표
            Q_extreme[old_position][action] += 0.1 * (reward - Q_extreme[old_position][action])
            break
        else:
            best_next = np.max(Q_extreme[position])
            Q_extreme[old_position][action] += 0.1 * (
                reward + 0.9 * best_next - Q_extreme[old_position][action]
            )

show_q_table(Q_extreme)
print("\n💡 함정(위치 2)으로 가는 행동의 Q값이 매우 낮음!")

## 🎓 다음 단계

### 이제 이해한 것:
✅ Agent, Environment, State, Action, Reward  
✅ Q-테이블과 Q-learning  
✅ 탐험 vs 활용 (ε-greedy)  
✅ 학습률과 할인율  

### 다음 도전:
1. **2D GridWorld** (01_rl_basics.ipynb)
   - 이제 2차원으로 확장!
   - 같은 원리, 더 큰 세계

2. **OpenAI Gym 환경**
   ```python
   import gym
   env = gym.make('FrozenLake-v1')
   ```

3. **추천 학습 자료**
   - 🎥 David Silver RL Course (Lecture 1-3)
   - 📖 Sutton & Barto Chapter 1-3
   - 🇰🇷 모두를 위한 RL (김성훈 교수)

In [ ]:
print("🎉 축하합니다! RL의 핵심을 이해하셨습니다!")
print("\n💪 이제 01_rl_basics.ipynb가 훨씬 쉬워질 거예요!")
print("\n다음 명령어로 계속하세요:")
print("jupyter notebook 01_rl_basics.ipynb")